# Astrora Quickstart Tutorial

Welcome to Astrora! This notebook will guide you through the essential features of the library, from creating your first orbit to advanced mission analysis.

**What you'll learn:**
- Creating orbits from vectors and classical elements
- Working with astropy units (poliastro-compatible)
- Propagating orbits in time
- Basic orbital maneuvers
- Visualization

**Prerequisites:** Basic Python knowledge and fundamental orbital mechanics concepts

## Setup

First, let's import the necessary libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.time import Time

# Import Astrora
from astrora import Orbit, bodies
from astrora.maneuvers import hohmann_transfer

# Set up matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ All imports successful!")

## 1. Creating Your First Orbit

### Method 1: From Position and Velocity Vectors

The most direct way to create an orbit is by specifying position and velocity vectors:

In [ ]:
# Define position and velocity vectors (SI units: meters and m/s)
r = np.array([7000e3, 0.0, 0.0])  # 7000 km from Earth's center
v = np.array([0.0, 7546.0, 0.0])   # Circular orbit velocity

# Create the orbit
orbit = Orbit.from_vectors(bodies.Earth, r, v)

# Display basic orbital properties
print("Orbit Properties:")
print(f"  Semi-major axis: {orbit.a.to_value('km'):.2f} km")
print(f"  Eccentricity: {orbit.ecc:.6f}")
print(f"  Period: {orbit.period.to_value('hour'):.2f} hours")
print(f"  Inclination: {np.rad2deg(orbit.inc):.2f} degrees")
print(f"  Altitude: {(orbit.a.to_value('m') - bodies.Earth.R.to_value('m'))/1000:.2f} km")

### Method 2: With Astropy Units (poliastro-compatible)

Astrora fully supports astropy units, making it compatible with poliastro/hapsira workflows:

In [ ]:
# Define vectors with units
r = [6800, 0, 0] * u.km
v = [0, 7.66, 0] * u.km / u.s
epoch = Time('2024-01-01 12:00:00', scale='utc')

# Create orbit with units
orbit_units = Orbit.from_vectors(bodies.Earth, r, v, epoch)

# Access properties with units
print("Orbit with Units:")
print(f"  Altitude: {(orbit_units.a - bodies.Earth.R).to(u.km):.2f}")
print(f"  Period: {orbit_units.period.to(u.minute):.2f}")
print(f"  Velocity magnitude: {np.linalg.norm(orbit_units.v.to_value('km/s')):.2f} km/s")

## 2. Creating Orbits from Classical Elements

For many applications, it's more convenient to work with classical orbital elements (Keplerian elements):

In [ ]:
# Create ISS-like orbit using classical elements
iss_orbit = Orbit.from_classical(
    bodies.Earth,
    a=6778 * u.km,        # Semi-major axis (~400 km altitude)
    ecc=0.0005 * u.one,   # Near-circular
    inc=51.6 * u.deg,     # Inclination (ISS standard)
    raan=45.0 * u.deg,    # Right ascension of ascending node
    argp=0.0 * u.deg,     # Argument of periapsis
    nu=0.0 * u.deg        # True anomaly (starting position)
)

print("ISS-like Orbit:")
print(f"  Semi-major axis: {iss_orbit.a.to(u.km):.2f}")
print(f"  Eccentricity: {iss_orbit.ecc:.6f}")
print(f"  Inclination: {iss_orbit.inc.to(u.deg):.2f}")
print(f"  Period: {iss_orbit.period.to(u.minute):.2f}")
print(f"  Orbital velocity: {np.linalg.norm(iss_orbit.v.to_value('km/s')):.2f} km/s")

### Common Orbit Types

Let's create several common orbit types:

In [ ]:
# Low Earth Orbit (LEO)
leo = Orbit.from_classical(
    bodies.Earth,
    a=6778 * u.km,
    ecc=0.0 * u.one,
    inc=28.5 * u.deg,  # Cape Canaveral latitude
    raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

# Geostationary Orbit (GEO)
geo = Orbit.from_classical(
    bodies.Earth,
    a=42164 * u.km,
    ecc=0.0 * u.one,
    inc=0.0 * u.deg,  # Equatorial
    raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

# Geostationary Transfer Orbit (GTO)
gto = Orbit.from_classical(
    bodies.Earth,
    a=24378 * u.km,       # Semi-major axis
    ecc=0.73 * u.one,     # Highly elliptical
    inc=28.5 * u.deg,
    raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

# Sun-Synchronous Orbit (SSO)
sso = Orbit.from_classical(
    bodies.Earth,
    a=7078 * u.km,        # ~700 km altitude
    ecc=0.001 * u.one,
    inc=98.2 * u.deg,     # Retrograde, sun-synchronous
    raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

print("Orbit Comparison:")
print(f"  LEO period: {leo.period.to(u.minute):.1f}")
print(f"  GEO period: {geo.period.to(u.hour):.2f}")
print(f"  GTO period: {gto.period.to(u.hour):.2f}")
print(f"  SSO period: {sso.period.to(u.minute):.1f}")
print(f"\n  GTO apogee: {gto.r_a.to(u.km):.0f}")
print(f"  GTO perigee: {gto.r_p.to(u.km):.0f}")

## 3. Propagating Orbits

Astrora provides high-performance orbit propagation using Rust-based numerical integrators:

In [ ]:
# Propagate ISS orbit for one full period
period = iss_orbit.period.to_value('s')
times = np.linspace(0, period, 100)  # 100 points over one orbit

# Propagate using Kepler's equations (very fast, no perturbations)
positions = []
for t in times:
    prop_orbit = iss_orbit.propagate(t * u.s)
    positions.append(prop_orbit.r.to_value('km'))

positions = np.array(positions)

# Plot the orbit
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot Earth
u_sphere = np.linspace(0, 2 * np.pi, 50)
v_sphere = np.linspace(0, np.pi, 50)
x_earth = 6371 * np.outer(np.cos(u_sphere), np.sin(v_sphere))
y_earth = 6371 * np.outer(np.sin(u_sphere), np.sin(v_sphere))
z_earth = 6371 * np.outer(np.ones(np.size(u_sphere)), np.cos(v_sphere))
ax.plot_surface(x_earth, y_earth, z_earth, color='lightblue', alpha=0.6)

# Plot orbit
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], 'r-', linewidth=2, label='ISS Orbit')
ax.scatter([positions[0, 0]], [positions[0, 1]], [positions[0, 2]],
           color='green', s=100, label='Start Position')

ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title('ISS Orbit Propagation (One Period)')
ax.legend()
ax.set_box_aspect([1,1,1])
plt.tight_layout()
plt.show()

print(f"Propagated {len(times)} points over {period/60:.2f} minutes")

## 4. Orbital Maneuvers

### Hohmann Transfer

The Hohmann transfer is the most fuel-efficient two-impulse transfer between circular coplanar orbits:

In [ ]:
# Calculate LEO to GEO transfer
r_leo = 6778e3  # meters
r_geo = 42164e3  # meters
mu = bodies.Earth.mu.to_value('m^3/s^2')

# Calculate Hohmann transfer parameters
dv1, dv2, transfer_time = hohmann_transfer(r_leo, r_geo, mu)

print("LEO to GEO Hohmann Transfer:")
print(f"  ΔV1 (at LEO): {dv1:.2f} m/s ({dv1/1000:.3f} km/s)")
print(f"  ΔV2 (at GEO): {dv2:.2f} m/s ({dv2/1000:.3f} km/s)")
print(f"  Total ΔV: {(dv1 + dv2)/1000:.3f} km/s")
print(f"  Transfer time: {transfer_time/3600:.2f} hours")

# Calculate transfer orbit properties
a_transfer = (r_leo + r_geo) / 2
ecc_transfer = (r_geo - r_leo) / (r_geo + r_leo)

print(f"\nTransfer Orbit:")
print(f"  Semi-major axis: {a_transfer/1000:.0f} km")
print(f"  Eccentricity: {ecc_transfer:.4f}")
print(f"  Perigee altitude: {(r_leo - 6371e3)/1000:.0f} km")
print(f"  Apogee altitude: {(r_geo - 6371e3)/1000:.0f} km")

### Visualize the Transfer

In [ ]:
# Create LEO, GEO, and transfer orbits for visualization
leo_orbit = Orbit.from_classical(
    bodies.Earth,
    a=r_leo * u.m,
    ecc=0.0 * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

geo_orbit = Orbit.from_classical(
    bodies.Earth,
    a=r_geo * u.m,
    ecc=0.0 * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

transfer_orbit = Orbit.from_classical(
    bodies.Earth,
    a=a_transfer * u.m,
    ecc=ecc_transfer * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

# Propagate all orbits
theta = np.linspace(0, 2*np.pi, 100)

def orbit_xy(orbit, theta):
    """Get x, y coordinates for orbit at given true anomalies"""
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            orbit.attractor, orbit.a, orbit.ecc, orbit.inc,
            orbit.raan, orbit.argp, angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)
    return positions[:, 0], positions[:, 1]

# Plot
fig, ax = plt.subplots(figsize=(12, 12))

# Earth
earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth')
ax.add_patch(earth_circle)

# Orbits
x_leo, y_leo = orbit_xy(leo_orbit, theta)
x_geo, y_geo = orbit_xy(geo_orbit, theta)
x_transfer, y_transfer = orbit_xy(transfer_orbit, theta[:50])  # Half orbit only

ax.plot(x_leo, y_leo, 'g-', linewidth=2, label='LEO (Initial)')
ax.plot(x_geo, y_geo, 'b-', linewidth=2, label='GEO (Target)')
ax.plot(x_transfer, y_transfer, 'r--', linewidth=2, label='Transfer Orbit')

# Mark burn locations
ax.scatter([x_leo[0]], [y_leo[0]], color='red', s=200, marker='*',
           zorder=5, label='ΔV1 Burn')
ax.scatter([x_transfer[-1]], [y_transfer[-1]], color='orange', s=200,
           marker='*', zorder=5, label='ΔV2 Burn')

ax.set_xlabel('X (km)', fontsize=12)
ax.set_ylabel('Y (km)', fontsize=12)
ax.set_title('Hohmann Transfer: LEO to GEO', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.set_xlim(-50000, 50000)
ax.set_ylim(-50000, 50000)
plt.tight_layout()
plt.show()

## 5. Working with Different Celestial Bodies

Astrora supports multiple celestial bodies:

In [ ]:
# Explore available bodies
print("Available Celestial Bodies:")
print(f"  Sun: μ = {bodies.Sun.mu.to(u.km**3/u.s**2):.2e}")
print(f"  Earth: μ = {bodies.Earth.mu.to(u.km**3/u.s**2):.2e}")
print(f"  Moon: μ = {bodies.Moon.mu.to(u.km**3/u.s**2):.2e}")
print(f"  Mars: μ = {bodies.Mars.mu.to(u.km**3/u.s**2):.2e}")
print(f"\n  Earth radius: {bodies.Earth.R.to(u.km):.0f}")
print(f"  Mars radius: {bodies.Mars.R.to(u.km):.0f}")
print(f"  Moon radius: {bodies.Moon.R.to(u.km):.0f}")

In [ ]:
# Create a lunar orbit
lunar_orbit = Orbit.from_classical(
    bodies.Moon,
    a=2000 * u.km,      # 2000 km from Moon's center (~260 km altitude)
    ecc=0.05 * u.one,
    inc=30 * u.deg,
    raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

print("Lunar Orbit Properties:")
print(f"  Altitude: {(lunar_orbit.a - bodies.Moon.R).to(u.km):.2f}")
print(f"  Period: {lunar_orbit.period.to(u.hour):.2f}")
print(f"  Velocity: {np.linalg.norm(lunar_orbit.v.to_value('km/s')):.2f} km/s")
print(f"  Eccentricity: {lunar_orbit.ecc:.4f}")

## 6. Performance Comparison

One of Astrora's key advantages is its Rust-powered performance. Let's see it in action:

In [ ]:
import time

# Benchmark: Create 1000 orbits and convert between representations
n_iterations = 1000

start_time = time.time()
for i in range(n_iterations):
    # Create orbit from vectors
    r_test = np.array([7000e3 + i*100, 0.0, 0.0])
    v_test = np.array([0.0, 7546.0, 0.0])
    orbit_test = Orbit.from_vectors(bodies.Earth, r_test, v_test)

    # Access classical elements (triggers conversion)
    _ = orbit_test.a
    _ = orbit_test.ecc
    _ = orbit_test.inc

elapsed = time.time() - start_time

print(f"Performance Benchmark:")
print(f"  Created {n_iterations} orbits in {elapsed:.3f} seconds")
print(f"  Average time per orbit: {elapsed/n_iterations*1000:.3f} ms")
print(f"  Rate: {n_iterations/elapsed:.0f} orbits/second")
print(f"\n  ⚡ Powered by Rust for maximum performance!")

## Summary

In this quickstart notebook, you learned:

✅ How to create orbits from position/velocity vectors and classical elements

✅ Working with astropy units (poliastro-compatible)

✅ Propagating orbits in time

✅ Calculating Hohmann transfers

✅ Working with different celestial bodies

✅ Visualizing orbits with matplotlib

✅ Experiencing Rust-powered performance

### Next Steps

Continue your journey with:
- **02_orbital_mechanics.ipynb** - Deep dive into orbital elements and conversions
- **03_maneuvers_transfers.ipynb** - Advanced orbital maneuvers
- **04_visualization_plotting.ipynb** - 2D/3D plots, animations, ground tracks
- **05_advanced_techniques.ipynb** - Monte Carlo, perturbations, mission analysis

For more information, see the [documentation](https://github.com/cachemcclure/astrora) and [examples](../examples/).